In [5]:
# Import necessary libraries
import cv2
import numpy as np
import os
import pandas as pd
import mahotas.features
import skimage.measure as measure
from skimage import feature, measure
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage import io, color, feature
from scipy.stats import moment
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import ast
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from skimage import feature, measure
import pandas as pd


In [12]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.stats import moment
import mahotas
import joblib
from skimage import measure

# Load the test image
image_path = r"C:\Users\User\Downloads\IMG_20240501_184017.jpg"
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Unable to load image at path {image_path}")
else:
    input_size = (224, 224)

    # Normalize the image
    normalized_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Resize the image to match the input size of the CNN
    resized_image = cv2.resize(normalized_image, input_size)

    # Convert the resized image to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Calculate color moments for each channel
    moments_r = moment(image[:, :, 0].ravel(), moment=[1, 2, 3])
    moments_g = moment(image[:, :, 1].ravel(), moment=[1, 2, 3])
    moments_b = moment(image[:, :, 2].ravel(), moment=[1, 2, 3])

    # Calculate Haralick texture features
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    entropy = graycoprops(glcm, 'ASM')[0, 0]

    # Calculate Histogram of Oriented Gradients (HOG)
    hog_features = feature.hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', feature_vector=True)

    # Calculate Local Binary Patterns (LBP)
    lbp = local_binary_pattern(gray_image, 8, 1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))

    # Calculate Zernike Moments
    zernike_moments = mahotas.features.zernike_moments(gray_image, radius=10, degree=8)

    # Calculate Hu Moments
    moments_hu = cv2.HuMoments(cv2.moments(gray_image)).flatten()

    # Apply Gabor filters
    gabor_features = []
    kernels = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        kernel = cv2.getGaborKernel((21, 21), 5.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        kernels.append(kernel)
        fimg = cv2.filter2D(gray_image, cv2.CV_8UC3, kernel)
        gabor_features.append(fimg.mean())
        gabor_features.append(fimg.var())

    # Feature extraction
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([hsv_image], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv_image], [2], None, [256], [0, 256])

    glcm = feature.graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    contrast = feature.graycoprops(glcm, 'contrast').ravel()[0]
    dissimilarity = feature.graycoprops(glcm, 'dissimilarity').ravel()[0]
    homogeneity = feature.graycoprops(glcm, 'homogeneity').ravel()[0]
    energy = feature.graycoprops(glcm, 'energy').ravel()[0]
    correlation = feature.graycoprops(glcm, 'correlation').ravel()[0]

    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea) if contours else None

    if largest_contour is not None:
        area = cv2.contourArea(largest_contour)
        perimeter = cv2.arcLength(largest_contour, True)
        x, y, w, h = cv2.boundingRect(largest_contour)
        aspect_ratio = float(w) / h
        contour_image = np.zeros_like(gray_image)
        cv2.drawContours(contour_image, [largest_contour], -1, (255), thickness=cv2.FILLED)
        eccentricity = measure.regionprops(measure.label(contour_image))[0].eccentricity
        hull = cv2.convexHull(largest_contour)
        hull_area = cv2.contourArea(hull)
        hull_perimeter = cv2.arcLength(hull, True)
        solidity = area / hull_area

    # Combine all features into a single feature vector
    features = np.concatenate((
                             hist_h.flatten().tolist(),
                             hist_s.flatten().tolist(),
                             hist_v.flatten().tolist(),
                             [contrast], [dissimilarity],
                             [homogeneity],  [energy],  [correlation],
                             [area],  [perimeter], [aspect_ratio],
                             [eccentricity],  [hull_area],
                             [hull_perimeter],  [solidity],
                             [moments_r[0]], [moments_r[1]],  [moments_r[2]],
                             [moments_g[0]], [moments_g[1]], [moments_g[2]],
                             [moments_b[0]],  [moments_b[1]], [moments_b[2]],
                             [contrast],  [correlation],  [entropy],
                             hog_features.tolist(),  hist.tolist(),
                             zernike_moments.tolist(),  moments_hu.tolist(),
                             gabor_features))

    print(f"Number of features: {features.size}")  # Print the number of features

    # Define paths to the scaler and model files
    scaler_path = r"C:\Users\User\Desktop\project\model_output\logistic_regression_scaler.pkl"
    model_path =  r"C:\Users\User\Desktop\project\model_output\logistic_regression_model.pkl"

    # Load the scaler and model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)

    # Standardize the features
    features_scaled = scaler.transform(features.reshape(1, -1))

    # Make a prediction
    prediction = model.predict(features_scaled)

    print(f"Predicted class: {prediction[0]}")


Number of features: 27085


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 27085 features, but StandardScaler is expecting 795 features as input.